In [1]:
from sklearn.tree._tree import TREE_LEAF
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
import pandas as pd
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, roc_auc_score, mean_squared_error, mean_absolute_error
import copy as copy
import importlib
import numpy as numpy
from statistics import mean
import os
import pickle
import pydotplus

import functions as func

labels_class = ["Prog1_scaled","Prog1_beste", "MfI1_beste", "beste", "beide"]
others_class = [["Prog1_beste","MfI1_beste","beste","beide"],["MfI1_beste","beste","beide","Prog1_scaled"],["Prog1_beste","beste","beide","Prog1_scaled"],["Prog1_beste","MfI1_beste","beide","Prog1_scaled"]
         ,["Prog1_beste","MfI1_beste","beste","Prog1_scaled"]]
labels_reg = ["Prog1_scaled","Prog1_beste", "MfI1_beste", "beste"]
others_reg = [["Prog1_beste","MfI1_beste","beste"],["MfI1_beste","beste","Prog1_scaled"],["Prog1_beste","beste","Prog1_scaled"],["Prog1_beste","MfI1_beste","Prog1_scaled"]]

measure_class = ["accuracy","f1","roc_auc"]
measure_reg = ["reg_accuracy","mse","mae"]

#Methods of Feature Engineering for Classification
methods_class = {"Prog1_beste": ["Basis"],
               "MfI1_beste": ["Korrelation"],
               "beste": ["RFECV"],
                "beide": ["RFE"]}

#Number of attributes for RFE for Classification
num_attributes_acc_noten = {"Prog1_beste": 7, "MfI1_beste": 48, "beste": 6}
num_attributes_acc_bestanden = {"Prog1_beste": 6, "MfI1_beste": 35, "beste": 28, "beide": 59}
num_attributes_f1 = {"Prog1_beste": 6, "MfI1_beste": 35, "beste": 3}
num_attributes_auc = {"Prog1_beste": 5, "MfI1_beste": 66, "beste": 4}

selector_acc = {}
selector_f1 = {}
selector_auc = {}

#Attributes with highest correlation
columns_reg = {"Prog1_beste": ["Abinote","SKMat_1","SKMat_2","SKMat_3","SKMat_4","mean_SKMat","SKInf_1", "mean_SKInf",
                               "Ktyp_exp", "BFI_K_3", "mean_BFI_K_G", "BM_Inf_17"],
               "Prog1_scaled": ["Abinote","SKMat_1","SKMat_2","SKMat_3","SKMat_4","mean_SKMat","SKInf_1",
                               "Ktyp_exp", "BFI_K_3", "mean_BFI_K_G", "BM_Inf_17"],
               "MfI1_beste": ["Abinote","SKMat_1","SKMat_2","SKMat_3","SKMat_4","mean_SKMat","SKInf_1", "SKInf_4",
                              "mean_SKInf", "Kurs_Inf", "Ktyp_exp", "BFI_K_7", "BM_Inf_14","LMI_3","LMI_6"],
               "beste": ["Abinote","SKMat_1","SKMat_2","SKMat_3","SKMat_4","mean_SKMat","SKInf_1","mean_SKInf",
                         "Ktyp_exp", "BFI_K_3","mean_BFI_K_G","BM_Inf_17","LMI_3"]}


columns_class = {"Prog1_beste": ["Abinote","SKMat_1","SKMat_3","mean_SKMat","SKInf_1"],
               "MfI1_beste": ["Abinote","SKMat_1","SKMat_2","SKMat_3","SKMat_4","mean_SKMat","SKInf_1","mean_SKInf", 
                              "BFI_K_11", "BM_Inf_18", "MfI1_beste"],
                "beste": ["Abinote","SKMat_1","SKMat_3","mean_SKMat","SKInf_1"],
               "beide": ["Abinote","SKMat_1","SKMat_3","SKMat_4","mean_SKMat","SKInf_1","SKInf_4","mean_SKInf","Kurs_Inf",
                         "BFI_K_7","BM_Inf_17","LMI_3","LMI_28"]}


selector = {}
selector_mae = {}
#Methods of Feature Engineering for Regression
methods_reg = {"Prog1_beste": ["Korrelation"],
               "Prog1_scaled": ["Korrelation"],
               "MfI1_beste": ["Korrelation"],
               "beste": ["RFE"]}
#Number of attributes for RFE for Regression
num_attributes_rmse = {"Prog1_beste": 8, "Prog1_scaled": 8, "MfI1_beste": 24, "beste": 27}
num_attributes_mae = {"Prog1_beste": 7, "Prog1_scaled": 7, "MfI1_beste": 4, "beste": 13}




In [2]:
other_count = 0
correlation = False
#Scenarios 4. - 7. 
for LABEL in labels_class:
    print("====================%s=============="%(LABEL))
    #Continuuos labels after normalization and scaling not suitable for accuracy
    if LABEL == "Prog1_scaled":
        other_count+=1
        continue
    for measure in measure_class:
        print("====================%s=============="%(measure))
        if LABEL == "beide" and measure != "accuracy":
            continue
        for method in methods_class[LABEL]:
            print("Methode: ", method)
            if method == "Korrelation":
                correlation = True
            #Load data
            dt = DecisionTreeClassifier(random_state=42)

            data_raw,train_raw,test_raw,validation_raw,names,feature_names = func.load_data_bestanden_raw(LABEL,others_class[other_count][0],
                                                                         others_class[other_count][1],others_class[other_count][2], correlation)
            correlation = False
            if method == "RFE":
                estimator = RandomForestClassifier(n_estimators = 100, verbose=1, n_jobs=-1, random_state=42)
                selector_acc[LABEL] = RFE(estimator, n_features_to_select= num_attributes_acc_bestanden[LABEL], step=1, verbose=1)
                selector_acc[LABEL] = selector_acc[LABEL].fit(train_raw[:,:-1], train_raw[:,-1])

            elif method == "RFECV":
                estimator = RandomForestClassifier(n_estimators = 100, verbose=1, n_jobs=-1, random_state=42)
                selector_acc[LABEL] = RFECV(estimator, step=1, cv=5,scoring='accuracy', verbose=1, n_jobs=-1)
                selector_acc[LABEL] = selector_acc[LABEL].fit(train_raw[:,:-1], train_raw[:,-1])

            if (method == "RFE" or method == "RFECV"):
                # Select attributes
                supp_acc = selector_acc[LABEL].get_support(indices=True) 
                feature_names = feature_names[supp_acc]
                supp_acc = numpy.append(supp_acc , -1)
                train_raw = copy.deepcopy(train_raw[:,supp_acc])
                test_raw = copy.deepcopy(test_raw[:,supp_acc])
                validation_raw = copy.deepcopy(validation_raw[:,supp_acc])
                data_raw = copy.deepcopy(data_raw[:,supp_acc])
              
            dt.fit(train_raw[:,:-1],train_raw[:,-1])
            #Results before pruning
            func.print_results_class(dt,train_raw,test_raw,validation_raw,LABEL,"vor")
        
            os.makedirs(os.path.dirname("graphs/dot/%s/REP/select/%s/prune_full_dt.dot"%(LABEL,measure)), exist_ok = True)

            tree.export_graphviz(dt, out_file='graphs/dot/%s/REP/select/%s/prune_full_dt.dot'%(LABEL,measure), feature_names= feature_names,
                     filled=True, rounded=True,special_characters=True)

            os.system("dot -T png graphs/dot/%s/REP/select/%s/prune_full_dt.dot -o graphs/%s/REP/select/%s/prune_full_dt.png"%(LABEL,measure,LABEL,measure))

            num_nodes = dt.tree_.node_count
            original_tree = copy.deepcopy(dt)
            best_tree = dt
            #Prune
            func.rep(dt,validation_raw,measure,LABEL,feature_names,select=True)

            #Render best pruned tree
            os.makedirs(os.path.dirname('graphs/dot/%s/REP/select/%s/best_pruned.dot'%(LABEL,measure)), exist_ok = True)
            os.makedirs(os.path.dirname('graphs/%s/REP/select/%s/best_pruned.png'%(LABEL,measure)), exist_ok = True)

            tree.export_graphviz(dt, out_file='graphs/dot/%s/REP/select/%s/best_pruned.dot'%(LABEL,measure), feature_names= feature_names,
                            filled=True, rounded=True,special_characters=True)
            os.system("dot -T png graphs/dot/%s/REP/select/%s/best_pruned.dot -o graphs/%s/REP/select/%s/best_pruned.png"%(LABEL,measure,LABEL,measure))
            #Results after pruning
            func.print_results_class(dt,train_raw,test_raw,validation_raw,LABEL,"nach")
            #Store model permanently
            os.makedirs(os.path.dirname("pruned_models/REP/select/%s/%s/%s_%s.pkl"%(LABEL,measure,LABEL,measure)), exist_ok = True)
            temp_pkl_file = open("pruned_models/REP/select/%s/%s/%s_%s.pkl"%(LABEL,measure,LABEL,measure), "wb")
            pickle.dump(copy.deepcopy(dt), temp_pkl_file)
            temp_pkl_file.close()


    other_count += 1

====================Prog1_scaled==============
====================Prog1_beste==============
====================accuracy==============
Methode:  Basis
[[83  0]
 [ 0 37]]
ACC vor Pruning Training:  1.0
AUC vor Pruning Training:  1.0
F1 vor Pruning Training:  1.0
[[60 36]
 [17 19]]
ACC vor Pruning Test:  0.5984848484848485
AUC vor Pruning Test:  0.5763888888888888
F1 vor Pruning Test:  0.4175824175824176
[[27 20]
 [ 5  5]]
ACC vor Pruning Validation:  0.5614035087719298
AUC vor Pruning Validation:  0.5372340425531915
F1 vor Pruning Validation:  0.28571428571428575
[[83  0]
 [19 18]]
ACC nach Pruning Training:  0.8416666666666667
AUC nach Pruning Training:  0.7657114946271573
F1 nach Pruning Training:  0.6545454545454547
[[79 17]
 [24 12]]
ACC nach Pruning Test:  0.6893939393939394
AUC nach Pruning Test:  0.579861111111111
F1 nach Pruning Test:  0.36923076923076925
[[39  8]
 [ 8  2]]
ACC nach Pruning Validation:  0.7192982456140351
AUC nach Pruning Validation:  0.4776595744680851
F1 nach

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.1s finished


Fitting estimator with 83 features.


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.1s finished


Fitting estimator with 82 features.


...

Fitting estimator with 61 features.


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.1s finished


Fitting estimator with 60 features.


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.1s finished


[[20  0  0]
 [ 0 12  0]
 [ 0  0 37]]
ACC vor Pruning Training:  1.0
[[11  4  7]
 [ 5  1 11]
 [11  8 21]]
ACC vor Pruning Test:  0.4177215189873418
[[11  3  0]
 [ 5  4  1]
 [ 4  0  4]]
ACC vor Pruning Validation:  0.59375
[[17  3  0]
 [ 1  9  2]
 [ 0  2 35]]
ACC nach Pruning Training:  0.8840579710144928
[[10  6  6]
 [ 5  2 10]
 [ 8 12 20]]
ACC nach Pruning Test:  0.4050632911392405
[[11  0  3]
 [ 4  6  0]
 [ 4  1  3]]
ACC nach Pruning Validation:  0.625
====================f1==============
====================roc_auc==============


In [3]:
other_count = 0
#Scenarios 1. - 3.
for LABEL in labels_reg:
    print("====================%s=============="%(LABEL))

    for measure in measure_reg:
        print("====================%s=============="%(measure))
        if measure == "reg_accuracy":
            #Continuuos labels after normalization and scaling not suitable for accuracy
            if LABEL == "Prog1_scaled":
                continue
        for method in methods_reg[LABEL]:
            print("Methode: ", method)
            if method == "Korrelation":
                correlation = True
            #Load data
            data_raw,train_raw,test_raw,validation_raw,names,feature_names = func.load_data_noten_raw(LABEL,others_reg[other_count][0],
                                                                         others_reg[other_count][1],
                                                                         others_reg[other_count][2], correlation)
            correlation = False
            if method == "RFE":
                if measure == "reg_accuracy":
                    estimator = RandomForestClassifier(n_estimators = 100, verbose=1, n_jobs=-1, random_state=42)
                    selector[LABEL] = RFE(estimator, n_features_to_select= num_attributes_acc_noten[LABEL], step=1, verbose=1)
                    selector[LABEL] = selector[LABEL].fit(train_raw[:,:-1], train_raw[:,-1])
                elif measure == "mse":
                    estimator = RandomForestRegressor(n_estimators = 100, verbose=1, n_jobs=-1, random_state=42)
                    selector[LABEL] = RFE(estimator, n_features_to_select= num_attributes_rmse[LABEL], step=1, verbose=1)
                    selector[LABEL] = selector[LABEL].fit(train_raw[:,:-1], train_raw[:,-1])
                elif measure == "mae":
                    estimator = RandomForestRegressor(n_estimators = 100, verbose=1, n_jobs=-1, random_state=42)
                    selector[LABEL] = RFE(estimator, n_features_to_select= num_attributes_mae[LABEL], step=1, verbose=1)
                    selector[LABEL] = selector[LABEL].fit(train_raw[:,:-1], train_raw[:,-1])
            elif method == "RFECV":
                if measure == "reg_accuracy":
                    estimator = RandomForestClassifier(n_estimators = 100, verbose=1, n_jobs=-1, random_state=42)
                    selector[LABEL] = RFECV(estimator, step=1, cv=5,scoring='accuracy', verbose=1, n_jobs=-1)
                    selector[LABEL] = selector[LABEL].fit(train_raw[:,:-1], train_raw[:,-1])
                elif measure == "mse":
                    estimator = RandomForestRegressor(n_estimators = 100, verbose=1, n_jobs=-1, random_state=42)
                    selector[LABEL] = RFECV(estimator, step=1, cv=5,scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)
                    selector[LABEL] = selector[LABEL].fit(train_raw[:,:-1], train_raw[:,-1])
                elif measure == "mae":
                    estimator = RandomForestRegressor(n_estimators = 100, verbose=1, n_jobs=-1, random_state=42)
                    selector[LABEL] = RFECV(estimator, step=1, cv=5,scoring='neg_mean_absolute_error', verbose=1, n_jobs=-1)
                    selector[LABEL] = selector[LABEL].fit(train_raw[:,:-1], train_raw[:,-1])
                    
            if (method == "RFE" or method == "RFECV"):
                # Select attributes
                supp = selector[LABEL].get_support(indices=True) 
                feature_names = feature_names[supp]
                supp = numpy.append(supp , -1)
                train_raw = copy.deepcopy(train_raw[:,supp])
                test_raw = copy.deepcopy(test_raw[:,supp])
                validation_raw = copy.deepcopy(validation_raw[:,supp])
                data_raw = copy.deepcopy(data_raw[:,supp])
              
            #Results before pruning
            if measure == "reg_accuracy":
                dt = DecisionTreeClassifier(random_state=42)
                dt.fit(train_raw[:,:-1],train_raw[:,-1])
                func.print_results_class(dt,train_raw,test_raw,validation_raw,LABEL,"vor", noten=True)

            else:
                dt = DecisionTreeRegressor(random_state=42)
                dt.fit(train_raw[:,:-1],train_raw[:,-1])
                func.print_results_reg(dt,train_raw,test_raw,validation_raw,"vor")

            os.makedirs(os.path.dirname("graphs/dot/%s/REP/select/%s/prune_full_dt.dot"%(LABEL,measure)), exist_ok = True)

            tree.export_graphviz(dt, out_file='graphs/dot/%s/REP/select/%s/prune_full_dt.png'%(LABEL,measure), feature_names= feature_names,
                         filled=True, rounded=True,special_characters=True)

            os.system("dot -T png graphs/dot/%s/REP/select/%s/prune_full_dt.dot -o graphs/%s/REP/select/%s/prune_full_dt.png"%(LABEL,measure,LABEL,measure))

            num_nodes = dt.tree_.node_count
            original_tree = copy.deepcopy(dt)
            best_tree = dt
            #Prune
            func.rep(dt,validation_raw,measure,LABEL,feature_names,select=True)

            #Render best pruned tree
            os.makedirs(os.path.dirname('graphs/dot/%s/REP/select/%s/best_pruned.dot'%(LABEL,measure)), exist_ok = True)
            os.makedirs(os.path.dirname('graphs/%s/REP/select/%s/best_pruned.png'%(LABEL,measure)), exist_ok = True)

            tree.export_graphviz(dt, out_file='graphs/dot/%s/REP/select/%s/best_pruned.dot'%(LABEL,measure), feature_names= feature_names,
                            filled=True, rounded=True,special_characters=True)
            os.system("dot -T png graphs/dot/%s/REP/select/%s/best_pruned.dot -o graphs/%s/REP/select/%s/best_pruned.png"%(LABEL,measure,LABEL,measure))
            #Results after pruning
            if measure == "reg_accuracy":
                func.print_results_class(dt,train_raw,test_raw,validation_raw,LABEL,"nach", noten=True)
            else:
                func.print_results_reg(dt,train_raw,test_raw,validation_raw,"nach")
            #Store model permanently
            os.makedirs(os.path.dirname("pruned_models/REP/select/%s/%s/%s_%s.pkl"%(LABEL,measure,LABEL,measure)), exist_ok = True)
            temp_pkl_file = open("pruned_models/REP/select/%s/%s/%s_%s.pkl"%(LABEL,measure,LABEL,measure), "wb")
            pickle.dump(copy.deepcopy(dt), temp_pkl_file)
            temp_pkl_file.close()
    other_count += 1

====================Prog1_scaled==============
====================reg_accuracy==============
====================mse==============
Methode:  Korrelation
RMSE vor Pruning Training:  1.9764975739480046e-14
MAE vor Pruning Training:  7.221909773299379e-15
RMSE vor Pruning Test:  161.9214328851352
MAE vor Pruning Test:  118.60402015102103
RMSE vor Pruning Validation:  146.630877451696
MAE vor Pruning Validation:  121.89934218350069
RMSE nach Pruning Training:  57.526608873894155
MAE nach Pruning Training:  37.657175235313055
RMSE nach Pruning Test:  138.35395752434718
MAE nach Pruning Test:  102.2863129839531
RMSE nach Pruning Validation:  120.13051643259433
MAE nach Pruning Validation:  95.58251717001461
====================mae==============
Methode:  Korrelation
RMSE vor Pruning Training:  1.9764975739480046e-14
MAE vor Pruning Training:  7.221909773299379e-15
RMSE vor Pruning Test:  161.9214328851352
MAE vor Pruning Test:  118.60402015102103
RMSE vor Pruning Validation:  146.6308774516

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.1s finished


Fitting estimator with 83 features.


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.1s finished


Fitting estimator with 82 features.


...

Fitting estimator with 15 features.


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.1s finished


Fitting estimator with 14 features.


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.1s finished


RMSE vor Pruning Training:  16.337966025741018
MAE vor Pruning Training:  3.1496062992125986
RMSE vor Pruning Test:  162.54096473625134
MAE vor Pruning Test:  117.6086956521739
RMSE vor Pruning Validation:  182.9845166491781
MAE vor Pruning Validation:  139.12280701754386
RMSE nach Pruning Training:  82.63976616065663
MAE nach Pruning Training:  60.706091695434615
RMSE nach Pruning Test:  145.66288239117108
MAE nach Pruning Test:  118.01654976083387
RMSE nach Pruning Validation:  147.43663963388374
MAE nach Pruning Validation:  117.23003629764067
